# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here
orders = pd.read_csv ('Orders.zip')
print(orders.head())

   Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75     

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [11]:
# Load the dataset
orders = pd.read_csv('Orders.zip')

# Step 1: Aggregate 'amount_spent' for each unique customer
customer_spending = orders.groupby('CustomerID')['amount_spent'].sum().reset_index()

# Step 2: Define quantile thresholds (for example, top 25% and top 50%)
quantile_75 = customer_spending['amount_spent'].quantile(0.75)
quantile_50 = customer_spending['amount_spent'].quantile(0.50)

# Step 3: Label customers as 'VIP' or 'Preferred'
customer_spending['customer_label'] = pd.cut(
    customer_spending['amount_spent'],
    bins=[0, quantile_50, quantile_75, float('inf')],
    labels=['Regular', 'Preferred', 'VIP']
)

# Step 4: Merge customer labels back into the 'orders' DataFrame
orders = orders.merge(customer_spending[['CustomerID', 'customer_label']], on='CustomerID', how='left')

# Display the resulting DataFrame with customer labels
print(customer_spending.head())


   CustomerID  amount_spent customer_label
0       12346      77183.60            VIP
1       12347       4310.00            VIP
2       12348       1797.24            VIP
3       12349       1757.55            VIP
4       12350        334.40        Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [21]:

# 1. Group the data by CustomerID and Country, summing the total amount_spent per customer
customer_spending = orders.groupby(['CustomerID', 'Country'])['amount_spent'].sum().reset_index()

# 2. Calculate the 90th percentile (Q90) to identify VIP customers
q90 = customer_spending['amount_spent'].quantile(0.90)

# 3. Function to label customers as VIP if their spending is greater than or equal to the 90th percentile
def label_customer(spent):
    if spent >= q90:
        return 'VIP'
    else:
        return 'Regular'

# Apply the function to label customers
customer_spending['label'] = customer_spending['amount_spent'].apply(label_customer)

# 4. Filter only VIP customers
vip_customers = customer_spending[customer_spending['label'] == 'VIP']

# 5. Count the number of VIPs by country
vip_count_by_country = vip_customers['Country'].value_counts().reset_index()
vip_count_by_country.columns = ['Country', 'VIP_Count']

# Display the country with the most VIP customers
top_country = vip_count_by_country.iloc[0]

print(vip_count_by_country)
print(f"The country with the most VIP customers is: {top_country['Country']}, with {top_country['VIP_Count']} VIP customers.")


            Country  VIP_Count
0    United Kingdom        356
1           Germany         23
2            France         18
3             Spain          6
4       Switzerland          5
5            Norway          4
6              EIRE          3
7             Japan          3
8           Belgium          2
9          Portugal          2
10          Finland          2
11        Australia          2
12          Iceland          1
13           Cyprus          1
14          Denmark          1
15           Israel          1
16        Singapore          1
17           Poland          1
18      Netherlands          1
19  Channel Islands          1
20           Sweden          1
The country with the most VIP customers is: United Kingdom, with 356 VIP customers.


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [26]:
# 1. Group the data by CustomerID and Country, summing the total amount_spent per customer
customer_spending = orders.groupby(['CustomerID', 'Country'])['amount_spent'].sum().reset_index()

# 2. Calculate the 75th and 90th percentiles to identify Preferred and VIP customers
q75 = customer_spending['amount_spent'].quantile(0.75)
q90 = customer_spending['amount_spent'].quantile(0.90)

# 3. Function to label customers as VIP, Preferred, or Regular
def label_customer(spent):
    if spent >= q90:
        return 'VIP'
    elif spent >= q75:
        return 'Preferred'
    else:
        return 'Regular'

# 4. Apply the function to label customers
customer_spending['label'] = customer_spending['amount_spent'].apply(label_customer)

# 5. Filter only VIP and Preferred customers
vip_preferred_customers = customer_spending[customer_spending['label'].isin(['VIP', 'Preferred'])]

# 6. Count the number of VIP and Preferred customers by country
vip_preferred_count_by_country = vip_preferred_customers.groupby('Country')['label'].count().reset_index()
vip_preferred_count_by_country.columns = ['Country', 'VIP_Preferred_Count']

# 7. Identify the country with the most VIP + Preferred customers
top_country = vip_preferred_count_by_country.loc[vip_preferred_count_by_country['VIP_Preferred_Count'].idxmax()]

# 8. Display the country with the most VIP + Preferred customers
print(vip_preferred_count_by_country)
print(f"The country with the most VIP + Preferred customers is: {top_country['Country']}, with {top_country['VIP_Preferred_Count']} customers.")

            Country  VIP_Preferred_Count
0         Australia                    4
1           Austria                    1
2           Belgium                   12
3            Canada                    1
4   Channel Islands                    4
5            Cyprus                    3
6           Denmark                    4
7              EIRE                    3
8           Finland                    5
9            France                   29
10          Germany                   39
11           Greece                    1
12          Iceland                    1
13           Israel                    2
14            Italy                    5
15            Japan                    4
16          Lebanon                    1
17            Malta                    1
18      Netherlands                    1
19           Norway                    7
20           Poland                    1
21         Portugal                    7
22        Singapore                    1
23            Sp